In [9]:
from nba_api.stats.endpoints import leaguegamelog
import model.dataset.data as data
import model.dataset.config as config

import pandas as pd
import numpy as np

In [11]:
season_games = pd.read_feather(config.SEASON_GAMES_DS)
season_games.set_index(["GAME_ID"], inplace=True)
season_games.sort_values(by=['GAME_DATE', 'GAME_ID'], inplace=True)

In [20]:
'-'.join(sorted(['BKN', 'ATL']))

'ATL-BKN'

In [34]:
#season_games["MATCHUP"] = '-'.join(sorted([season_games.TEAM_ABBREVIATION_HOME, season_games.TEAM_ABBREVIATION_AWAY]))


#.reset_index(level=0)
#.expanding().mean()

,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,...,OREB_AWAY_L10M,DREB_AWAY_L10M,REB_AWAY_L10M,AST_AWAY_L10M,STL_AWAY_L10M,BLK_AWAY_L10M,TOV_AWAY_L10M,PF_AWAY_L10M,PTS_AWAY_L10M,PLUS_MINUS_AWAY_L10M
GAME_ID,,,,,,,,,,,,,,,,,,,,,
0020200001,22002,1610612753,ORL,Orlando Magic,2002-10-29,ORL vs. PHI,W,240,36,81,...,11.0,35.0,46.0,19.0,10.0,1.0,20.0,22.0,88.0,-7.0
0020200002,22002,1610612758,SAC,Sacramento Kings,2002-10-29,SAC vs. CLE,W,240,36,80,...,17.0,29.0,46.0,14.0,7.0,5.0,24.0,20.0,67.0,-27.0
0020200003,22002,1610612747,LAL,Los Angeles Lakers,2002-10-29,LAL vs. SAS,L,240,27,85,...,11.0,41.0,52.0,15.0,15.0,11.0,20.0,26.0,87.0,5.0
0020200004,22002,1610612738,BOS,Boston Celtics,2002-10-30,BOS vs. CHI,L,240,36,77,...,12.0,35.0,47.0,29.0,6.0,5.0,18.0,19.0,99.0,3.0
0020200005,22002,1610612755,PHI,Philadelphia 76ers,2002-10-30,PHI vs. MIL,W,240,31,86,...,6.0,29.0,35.0,19.0,11.0,6.0,17.0,25.0,93.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0021901314,22019,1610612762,UTA,Utah Jazz,2020-08-13,UTA vs. SAS,W,240,47,101,...,7.5,31.5,40.5,24.0,7.0,4.5,15.0,19.5,111.5,-6.5
0021901315,22019,1610612745,HOU,Houston Rockets,2020-08-14,HOU vs. PHI,L,240,35,80,...,8.0,33.5,42.5,23.0,10.0,5.0,14.5,23.0,111.0,-0.5
0021901316,22019,1610612754,IND,Indiana Pacers,2020-08-14,IND vs. MIA,W,240,43,89,...,6.5,33.0,41.5,25.5,8.5,4.5,16.5,19.5,108.5,3.0


# Finding Games

To find a single game, use the `LeagueGameFinder` class.
While you can call it without any arguments and get ~30,000 games returned (I believe that's the max number of rows that nba.com will send in a response) across the NBA, WNBA, G-League, and international ball, it's a better idea to pass a team ID.

See `nba_api.stats.static.teams` in the [Basics Notebook](Basics.ipynb) for more detail on getting a team ID.

Let's try to find the last time the Celtics played the Raptors in 2017-18.
That will be four steps:
1. Fetch all Celtics games.
2. Select just games from the 2017-18 season (SEASON_ID ending in 2017).
3. Select games where the opponent is the Raptors (MATCHUP contains 'TOR').
4. Order by date and select the last row.

And last, we'll also get the play-by-play data from that game.

### Get All Celtics Games

In [334]:
import numpy as np
next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
                                       , season="2017-18").get_data_frames()[0].set_index('GAME_ID').sort_values(by=['GAME_DATE'])
next_season.dropna(inplace=True)
next_season["W_L"] = np.where(next_season['WL'] == 'W', 1, -1)

season_games_10lm = next_season.groupby(by=["GAME_ID", "TEAM_ID", "GAME_ID"])[['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
#.reset_index(level=0)

#print(next_season[(next_season.index == '0021700017') & (next_season.TEAM_ID == 1610612737)].iloc[0])
season_games_10lm

#.reset_index(level=0)
#.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})
#next_season
# result
# compare_df = pd.DataFrame()
# print(result["TEAM_ID"].shape)
# print(next_season["TEAM_ID"].shape)
# compare_df['team_id'] = next_season["TEAM_ID"]
# compare_df['result_team_id'] = result["TEAM_ID"]

# comparison = result["TEAM_ID"].values == next_season["TEAM_ID"].values
# equal_arrays = comparison.all()

# np.setdiff1d(result["TEAM_ID"].values, next_season["TEAM_ID"].values)
# diff = result["TEAM_ID"].values - next_season["TEAM_ID"].values
# diff
# #compare_df['different'] = np.where( == , True, False)
# #compare_df

MIN   FGM   FGA  FG_PCT  FG3M  \
GAME_ID    TEAM_ID    GAME_ID    GAME_ID                                       
0021700001 1610612738 0021700001 0021700001  240.0  36.0  88.0   0.409   8.0   
           1610612739 0021700001 0021700001  240.0  38.0  83.0   0.458   5.0   
0021700002 1610612744 0021700002 0021700002  240.0  43.0  80.0   0.538  16.0   
           1610612745 0021700002 0021700002  240.0  47.0  97.0   0.485  15.0   
0021700003 1610612765 0021700003 0021700003  240.0  41.0  96.0   0.427   9.0   
...                                            ...   ...   ...     ...   ...   
0021701228 1610612747 0021701228 0021701228  240.0  42.0  88.0   0.477  17.0   
0021701229 1610612757 0021701229 0021701229  240.0  41.0  89.0   0.461   9.0   
           1610612762 0021701229 0021701229  240.0  33.0  89.0   0.371   8.0   
0021701230 1610612745 0021701230 0021701230  240.0  31.0  82.0   0.378  13.0   
           1610612758 0021701230 0021701230  240.0  38.0  80.0   0.475   7.0   

                                             FG3A  FG3_PCT   FTM   FTA  \
GAME_ID    TEAM_ID    GAME_ID    GAME_ID                                 
0021700001 1610612738 0021700001 0021700001  32.0    0.250  19.0  25.0   
           1610612739 0021700001 0021700001  22.0    0.227  21.0  25.0   
0021700002 1610612744 0021700002 0021700002  30.0    0.533  19.0  21.0   
           1610612745 0021700002 0021700002  41.0    0.366  13.0  19.0   
0021700003 1610612765 0021700003 0021700003  26.0    0.346  11.0  12.0   
...                                           ...      ...   ...   ...   
0021701228 1610612747 0021701228 0021701228  39.0    0.436  14.0  16.0   
0021701229 1610612757 0021701229 0021701229  24.0    0.375  11.0  16.0   
           1610612762 0021701229 0021701229  23.0    0.348  19.0  24.0   
0021701230 1610612745 0021701230 0021701230  46.0    0.283   8.0  12.0   
           1610612758 0021701230 0021701230  26.0    0.269  13.0  20.0   

                                             FT_PCT  OREB  DREB   REB   AST  \
GAME_ID    TEAM_ID    GAME_ID    GAME_ID                                      
0021700001 1610612738 0021700001 0021700001   0.760   9.0  37.0  46.0  24.0   
           1610612739 0021700001 0021700001   0.840   9.0  41.0  50.0  19.0   
0021700002 1610612744 0021700002 0021700002   0.905   6.0  35.0  41.0  34.0   
           1610612745 0021700002 0021700002   0.684  10.0  33.0  43.0  28.0   
0021700003 1610612765 0021700003 0021700003   0.917   9.0  38.0  47.0  24.0   
...                                             ...   ...   ...   ...   ...   
0021701228 1610612747 0021701228 0021701228   0.875  10.0  39.0  49.0  25.0   
0021701229 1610612757 0021701229 0021701229   0.688   7.0  39.0  46.0  19.0   
           1610612762 0021701229 0021701229   0.792  14.0  39.0  53.0  18.0   
0021701230 1610612745 0021701230 0021701230   0.667   8.0  32.0  40.0  11.0   
           1610612758 0021701230 0021701230   0.650   6.0  42.0  48.0  22.0   

                                              STL  BLK   TOV    PF    PTS  \
GAME_ID    TEAM_ID    GAME_ID    GAME_ID                                    
0021700001 1610612738 0021700001 0021700001  11.0  4.0  12.0  24.0   99.0   
           1610612739 0021700001 0021700001   3.0  4.0  17.0  25.0  102.0   
0021700002 1610612744 0021700002 0021700002   5.0  9.0  17.0  25.0  121.0   
           1610612745 0021700002 0021700002   9.0  5.0  13.0  16.0  122.0   
0021700003 1610612765 0021700003 0021700003  14.0  3.0   9.0  21.0  102.0   
...                                           ...  ...   ...   ...    ...   
0021701228 1610612747 0021701228 0021701228   8.0  6.0  17.0  23.0  115.0   
0021701229 1610612757 0021701229 0021701229   9.0  9.0  10.0  22.0  102.0   
           1610612762 0021701229 0021701229   5.0  7.0  15.0  19.0   93.0   
0021701230 1610612745 0021701230 0021701230   2.0  6.0  14.0  24.0   83.0   
           1610612758 0021701230 0021701230   6.0  3.0  11.0  14.0   96.0   

             

In [330]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog
import model.dataset.data as data

seasons = data.load_seasons().SEASON.unique()
# season_games = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                             , season=seasons[-11]).get_data_frames()[0].sort_values(by=['SEASON_ID', 'GAME_DATE', 'GAME_ID'])
season_games = pd.DataFrame()
for season in seasons[-11:]:
    next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
                                           , season=season).get_data_frames()[0] \
                                            .set_index('GAME_ID').sort_values(by=['GAME_DATE'])
                                            #.sort_values(by=['SEASON_ID', 'GAME_DATE', 'GAME_ID'])
    next_season.dropna(inplace=True)
    next_season["W_L"] = np.where(next_season['WL'] == 'W', 1, -1)
    season_games_sum = next_season.groupby(by=["TEAM_ID"])[['W_L']]\
    .expanding().sum().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_sum, suffixes=['', '_CUM'], on=['GAME_ID', 'TEAM_ID'])
    
    season_games_mean = next_season.groupby(by=["TEAM_ID"])[['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
    .expanding().mean().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'], on=['GAME_ID', 'TEAM_ID'])
    
    next_season.groupby(by=["TEAM_ID"])[['W_L', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
    .rolling(window=10, min_periods=1).median().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_L10'], on=['GAME_ID', 'TEAM_ID'])
    
    season_home_rows = next_season[next_season.MATCHUP.str.contains('vs.')]
    season_away_rows = next_season[next_season.MATCHUP.str.contains('@')]
    # Join every row to all others with the same game ID.
    joined = pd.merge(season_home_rows, season_away_rows, suffixes=['_HOME', '_AWAY'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_HOME != joined.TEAM_ID_AWAY]
    season_games = pd.concat([season_games, result])
season_games.describe()

,TEAM_ID_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,W_L_HOME,W_L_CUM_HOME,MIN_MEAN_HOME,FGM_MEAN_HOME,FGA_MEAN_HOME,FG_PCT_MEAN_HOME,FG3M_MEAN_HOME,FG3A_MEAN_HOME,FG3_PCT_MEAN_HOME,FTM_MEAN_HOME,FTA_MEAN_HOME,FT_PCT_MEAN_HOME,OREB_MEAN_HOME,DREB_MEAN_HOME,REB_MEAN_HOME,AST_MEAN_HOME,STL_MEAN_HOME,BLK_MEAN_HOME,TOV_MEAN_HOME,PF_MEAN_HOME,PTS_MEAN_HOME,PLUS_MINUS_MEAN_HOME,MIN_L10_HOME,FGM_L10_HOME,FGA_L10_HOME,FG_PCT_L10_HOME,FG3M_L10_HOME,FG3A_L10_HOME,FG3_PCT_L10_HOME,FTM_L10_HOME,FTA_L10_HOME,FT_PCT_L10_HOME,OREB_L10_HOME,DREB_L10_HOME,REB_L10_HOME,AST_L10_HOME,STL_L10_HOME,BLK_L10_HOME,TOV_L10_HOME,PF_L10_HOME,PTS_L10_HOME,PLUS_MINUS_L10_HOME,TEAM_ID_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,W_L_AWAY,W_L_CUM_AWAY,MIN_MEAN_AWAY,FGM_MEAN_AWAY,FGA_MEAN_AWAY,FG_PCT_MEAN_AWAY,FG3M_MEAN_AWAY,FG3A_MEAN_AWAY,FG3_PCT_MEAN_AWAY,FTM_MEAN_AWAY,FTA_MEAN_AWAY,FT_PCT_MEAN_AWAY,OREB_MEAN_AWAY,DREB_MEAN_AWAY,REB_MEAN_AWAY,AST_MEAN_AWAY,STL_MEAN_AWAY,BLK_MEAN_AWAY,TOV_MEAN_AWAY,PF_MEAN_AWAY,PTS_MEAN_AWAY,PLUS_MINUS_MEAN_AWAY,MIN_L10_AWAY,FGM_L10_AWAY,FGA_L10_AWAY,FG_PCT_L10_AWAY,FG3M_L10_AWAY,FG3A_L10_AWAY,FG3_PCT_L10_AWAY,FTM_L10_AWAY,FTA_L10_AWAY,FT_PCT_L10_AWAY,OREB_L10_AWAY,DREB_L10_AWAY,REB_L10_AWAY,AST_L10_AWAY,STL_L10_AWAY,BLK_L10_AWAY,TOV_L10_AWAY,PF_L10_AWAY,PTS_L10_AWAY,PLUS_MINUS_L10_AWAY
count,1.311800e+04,13118.000000,13118.000000,13118.000000,13118.000000,13118.00000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,1.311800e+04,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000,13118.000000
mean,1.610613e+09,241.770468,38.882299,84.254536,0.462583,8.64301,24.000381,0.357804,17.957463,23.584388,0.762050,10.739747,32.948620,43.688367,23.086675,7.645601,5.106724,14.212685,20.002592,104.365071,2.708340,0.742491,0.173045,0.096432,241.717934,38.085095,84.095516,0.453950,8.404033,23.730823,0.351864,17.843699,23.499526,0.759425,10.689661,32.541448,43.231110,22.104230,7.697465,5.006291,14.714480,20.805515,102.417921,0.143803,241.717934,38.085095,84.095516,0.453950,8.404033,23.730823,0.351864,17.843699,23.499526,0.759425,10.689661,32.541448,43.231110,22.104230,7.697465,5.006291,14.714480,20.805515,102.417921,0.143803,1.610613e+09,241.770468,37.96

In [331]:
pd.options.display.max_rows = 999


season_games[season_games.isnull().any(axis=1)]

,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,W_L_HOME,W_L_CUM_HOME,MIN_MEAN_HOME,FGM_MEAN_HOME,FGA_MEAN_HOME,FG_PCT_MEAN_HOME,FG3M_MEAN_HOME,FG3A_MEAN_HOME,FG3_PCT_MEAN_HOME,FTM_MEAN_HOME,FTA_MEAN_HOME,FT_PCT_MEAN_HOME,OREB_MEAN_HOME,DREB_MEAN_HOME,REB_MEAN_HOME,AST_MEAN_HOME,STL_MEAN_HOME,BLK_MEAN_HOME,TOV_MEAN_HOME,PF_MEAN_HOME,PTS_MEAN_HOME,PLUS_MINUS_MEAN_HOME,MIN_L10_HOME,FGM_L10_HOME,FGA_L10_HOME,FG_PCT_L10_HOME,FG3M_L10_HOME,FG3A_L10_HOME,FG3_PCT_L10_HOME,FTM_L10_HOME,FTA_L10_HOME,FT_PCT_L10_HOME,OREB_L10_HOME,DREB_L10_HOME,REB_L10_HOME,AST_L10_HOME,STL_L10_HOME,BLK_L10_HOME,TOV_L10_HOME,PF_L10_HOME,PTS_L10_HOME,PLUS_MINUS_L10_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,W_L_AWAY,W_L_CUM_AWAY,MIN_MEAN_AWAY,FGM_MEAN_AWAY,FGA_MEAN_AWAY,FG_PCT_MEAN_AWAY,FG3M_MEAN_AWAY,FG3A_MEAN_AWAY,FG3_PCT_MEAN_AWAY,FTM_MEAN_AWAY,FTA_MEAN_AWAY,FT_PCT_MEAN_AWAY,OREB_MEAN_AWAY,DREB_MEAN_AWAY,REB_MEAN_AWAY,AST_MEAN_AWAY,STL_MEAN_AWAY,BLK_MEAN_AWAY,TOV_MEAN_AWAY,PF_MEAN_AWAY,PTS_MEAN_AWAY,PLUS_MINUS_MEAN_AWAY,MIN_L10_AWAY,FGM_L10_AWAY,FGA_L10_AWAY,FG_PCT_L10_AWAY,FG3M_L10_AWAY,FG3A_L10_AWAY,FG3_PCT_L10_AWAY,FTM_L10_AWAY,FTA_L10_AWAY,FT_PCT_L10_AWAY,OREB_L10_AWAY,DREB_L10_AWAY,REB_L10_AWAY,AST_L10_AWAY,STL_L10_AWAY,BLK_L10_AWAY,TOV_L10_AWAY,PF_L10_AWAY,PTS_L10_AWAY,PLUS_MINUS_L10_AWAY
GAME_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [176]:
#season_games['SEASON_YEAR'] = season_games.SEASON_ID.str[1:]
season_games.drop(columns=['VIDEO_AVAILABLE_HOME', 'VIDEO_AVAILABLE_AWAY'], axis=1, inplace=True)

KeyError: "['VIDEO_AVAILABLE_HOME' 'VIDEO_AVAILABLE_AWAY'] not found in axis"

In [194]:
def create_filter(team, season_id, game_date):
    return (season_games.GAME_DATE < game_date) & \
            (season_games.SEASON_ID == season_id) & \
            ((season_games.TEAM_ID_HOME == team) | (season_games.TEAM_ID_AWAY == team))
    

In [203]:
row.index

Index(['SEASON_ID', 'TEAM_ID_HOME', 'TEAM_ABBREVIATION_HOME', 'TEAM_NAME_HOME',
       'GAME_ID', 'GAME_DATE', 'MATCHUP_HOME', 'WL_HOME', 'MIN_HOME',
       'FGM_HOME', 'FGA_HOME', 'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME',
       'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME', 'FT_PCT_HOME', 'OREB_HOME',
       'DREB_HOME', 'REB_HOME', 'AST_HOME', 'STL_HOME', 'BLK_HOME', 'TOV_HOME',
       'PF_HOME', 'PTS_HOME', 'PLUS_MINUS_HOME', 'TEAM_ID_AWAY',
       'TEAM_ABBREVIATION_AWAY', 'TEAM_NAME_AWAY', 'MATCHUP_AWAY', 'WL_AWAY',
       'MIN_AWAY', 'FGM_AWAY', 'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY',
       'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY', 'FT_PCT_AWAY',
       'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY', 'STL_AWAY',
       'BLK_AWAY', 'TOV_AWAY', 'PF_AWAY', 'PTS_AWAY', 'PLUS_MINUS_AWAY',
       'SEASON_YEAR'],
      dtype='object')

In [199]:


row = season_games.iloc[200]
print(row)
home = row.TEAM_ID_HOME
away = row.TEAM_ID_AWAY
season_id = row.SEASON_ID
game_date = row.GAME_DATE

home_games = season_games[create_filter(home, season_id, game_date)]
away_games = season_games[create_filter(home, season_id, game_date)]

home_games.columns

#season_games.expanding().mean()


SEASON_ID                                  22009
TEAM_ID_HOME                          1610612746
TEAM_ABBREVIATION_HOME                       LAC
TEAM_NAME_HOME              Los Angeles Clippers
GAME_ID                               0020900201
GAME_DATE                             2009-11-23
MATCHUP_HOME                         LAC vs. MIN
WL_HOME                                        W
MIN_HOME                                     240
FGM_HOME                                      35
FGA_HOME                                      83
FG_PCT_HOME                                0.422
FG3M_HOME                                      2
FG3A_HOME                                     14
FG3_PCT_HOME                               0.143
FTM_HOME                                      19
FTA_HOME                                      22
FT_PCT_HOME                                0.864
OREB_HOME                                     16
DREB_HOME                                     28
REB_HOME            

Index(['SEASON_ID', 'TEAM_ID_HOME', 'TEAM_ABBREVIATION_HOME', 'TEAM_NAME_HOME',
       'GAME_ID', 'GAME_DATE', 'MATCHUP_HOME', 'WL_HOME', 'MIN_HOME',
       'FGM_HOME', 'FGA_HOME', 'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME',
       'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME', 'FT_PCT_HOME', 'OREB_HOME',
       'DREB_HOME', 'REB_HOME', 'AST_HOME', 'STL_HOME', 'BLK_HOME', 'TOV_HOME',
       'PF_HOME', 'PTS_HOME', 'PLUS_MINUS_HOME', 'TEAM_ID_AWAY',
       'TEAM_ABBREVIATION_AWAY', 'TEAM_NAME_AWAY', 'MATCHUP_AWAY', 'WL_AWAY',
       'MIN_AWAY', 'FGM_AWAY', 'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY',
       'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY', 'FT_PCT_AWAY',
       'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY', 'STL_AWAY',
       'BLK_AWAY', 'TOV_AWAY', 'PF_AWAY', 'PTS_AWAY', 'PLUS_MINUS_AWAY',
       'SEASON_YEAR'],
      dtype='object')

In [111]:
from nba_api.stats.endpoints import teamgamelogs

# Query for games where the Celtics were playing
gamefinder = teamgamelogs.TeamGameLogs(team_id_nullable=celtics_id)
all_games = teamgamelogs.TeamGameLogs(season_type_nullable="Regular Season"
                                          , season_nullable='2010-11').get_data_frames()[0]
teamgamelogs.SeasonNullable.current_season
# The first DataFrame of those returned is what we want.
all_games.head()


,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,2010-11,1610612758,SAC,Sacramento Kings,0021001230,2011-04-13T00:00:00,SAC vs. LAL,L,53.0,45,99,0.455,7,24,0.292,11,19,0.579,16,37,53,24,12.0,5,6,4,20,17,108,-8.0,1,1231,1231,1231,19,138,29,1283,805,331,1737,2163,1802,2350,216,229,85,653,567,1792,585,813,988,1881,565,1752
1,2010-11,1610612746,LAC,Los Angeles Clippers,0021001228,2011-04-13T00:00:00,LAC vs. MEM,W,48.0,44,81,0.543,3,15,0.200,19,27,0.704,12,25,37,36,18.0,16,7,4,26,24,110,7.0,1,1,1,1,163,191,1167,171,2068,1579,2211,1021,786,1811,792,2053,1772,5,2008,6,368,813,2140,450,437,710
2,2010-11,1610612761,TOR,Toronto Raptors,0021001221,2011-04-13T00:00:00,TOR vs. MIA,L,48.0,28,64,0.438,1,8,0.125,22,34,0.647,10,23,33,16,17.0,8,8,1,15,18,79,-18.0,1,1231,1231,1231,163,2374,2435,1583,2390,2383,2387,623,222,2158,1265,2252,2197,2067,1815,800,232,38,153,1695,2356,2260
3,2010-11,1610612754,IND,Indiana Pacers,0021001219,2011-04-13T00:00:00,IND @ ORL,L,48.0,27,88,0.307,9,25,0.360,11,16,0.688,9,26,35,16,18.0,11,1,4,23,18,74,-18.0,1,1231,1231,1231,163,2415,376,2455,388,261,1169,2163,2121,1921,1507,1915,2006,2067,2008,174,2288,813,1656,1695,2429,2260
4,2010-11,1610612742,DAL,Dallas Mavericks,0021001223,2011-04-13T00:00:00,DAL vs. NOH,W,48.0,40,73,0.548,13,28,0.464,28,34,0.824,6,36,42,32,14.0,7,1,1,17,28,121,32.0,1,1,1,1,163,617,2119,136,55,116,405,151,222,670,2164,316,1050,43,1071,1130,2288,38,388,95,93,20


In [112]:
all_games.groupby(all_games.SEASON_YEAR)[['GAME_ID']].count()
#.loc['2015':]

,GAME_ID
SEASON_YEAR,
2010-11,2460


In [35]:
from nba_api.stats.endpoints import leaguegamefinder


# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42019,1610612738,BOS,Boston Celtics,0041900306,2020-09-27,BOS @ MIA,L,240,113,43,99,0.434,15,46,0.326,12,15,0.800,13,29,42,26,7,2,10,21,-12.0
1,42019,1610612738,BOS,Boston Celtics,0041900305,2020-09-25,BOS vs. MIA,W,238,121,42,93,0.452,12,38,0.316,25,29,0.862,13,37,50,29,7,4,11,23,13.0
2,42019,1610612738,BOS,Boston Celtics,0041900304,2020-09-23,BOS @ MIA,L,239,109,39,82,0.476,14,40,0.350,17,21,0.810,12,34,46,28,3,5,19,24,0.2
3,42019,1610612738,BOS,Boston Celtics,0041900303,2020-09-19,BOS @ MIA,W,240,117,41,85,0.482,9,26,0.346,26,30,0.867,10,40,50,27,8,3,14,29,8.4
4,42019,1610612738,BOS,Boston Celtics,0041900302,2020-09-17,BOS vs. MIA,L,240,101,36,72,0.500,10,28,0.357,19,24,0.792,6,35,41,19,5,3,20,22,-5.0


As you can see above, the season ID is 5 digits.
I believe the last 4 will always be the current season (2018 for the 2018-19 season).
We can do a sanity check and look at how many games the Celtics have played in recent years.

In [24]:
# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count()

,GAME_ID
SEASON_ID,
1983,105
1984,103
1985,100
1986,105
1987,99
1988,85
1989,87
1990,93
1991,92


Note that some of these games are preseason and summer league, so these numbers aren't just regular season and playoffs.

### Filter to Games in the 2017-18 Season

In [11]:
# Subset the games to when the last 4 digits of SEASON_ID were 2017.
games_1718 = games[games.SEASON_ID.str[-4:] == '2017']
games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
203,42017,1610612738,BOS,Boston Celtics,0041700307,2018-05-27,BOS vs. CLE,L,240,79,29,85,0.341,7,39,0.179,14,19,0.737,11,31,42,18,6,0,5,20,-8.0
204,42017,1610612738,BOS,Boston Celtics,0041700306,2018-05-25,BOS @ CLE,L,241,99,38,74,0.514,12,28,0.429,11,20,0.550,5,26,31,25,5,2,13,18,-10.0
205,42017,1610612738,BOS,Boston Celtics,0041700305,2018-05-23,BOS vs. CLE,W,240,96,31,85,0.365,13,39,0.333,21,23,0.913,7,38,45,18,10,6,8,19,13.0
206,42017,1610612738,BOS,Boston Celtics,0041700304,2018-05-21,BOS @ CLE,L,239,102,35,85,0.412,9,28,0.321,23,30,0.767,9,28,37,21,9,3,9,26,-9.0
207,42017,1610612738,BOS,Boston Celtics,0041700303,2018-05-19,BOS @ CLE,L,240,86,29,74,0.392,6,22,0.273,22,28,0.786,6,28,34,16,4,4,15,25,-30.0


### Filter to Games Against the Raptors

In [12]:
# Subset the games to where MATCHUP contains 'TOR'.
raps_games_1718 = games_1718[games_1718.MATCHUP.str.contains('TOR')]
raps_games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
226,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,25,75,0.333,3,22,0.136,25,28,0.893,8,35,43,10,7,3,16,16,-18.0
228,22017,1610612738,BOS,Boston Celtics,0021701140,2018-03-31,BOS vs. TOR,W,240,110,40,88,0.455,7,17,0.412,23,27,0.852,8,33,41,19,6,3,5,23,11.0
249,22017,1610612738,BOS,Boston Celtics,0021700798,2018-02-06,BOS @ TOR,L,240,91,33,83,0.398,10,23,0.435,15,20,0.750,9,29,38,21,3,3,17,19,-20.0
290,22017,1610612738,BOS,Boston Celtics,0021700188,2017-11-12,BOS vs. TOR,W,239,95,35,87,0.402,10,26,0.385,15,19,0.789,15,31,46,24,9,2,14,18,1.0


### Sort by Game Date and Select the Last Row

In [13]:
last_raps_game = raps_games_1718.sort_values('GAME_DATE').iloc[-1]
last_raps_game

SEASON_ID                     22017
TEAM_ID                  1610612738
TEAM_ABBREVIATION               BOS
TEAM_NAME            Boston Celtics
GAME_ID                  0021701171
GAME_DATE                2018-04-04
MATCHUP                   BOS @ TOR
WL                                L
MIN                             237
PTS                              78
FGM                              25
FGA                              75
FG_PCT                        0.333
FG3M                              3
FG3A                             22
FG3_PCT                       0.136
FTM                              25
FTA                              28
FT_PCT                        0.893
OREB                              8
DREB                             35
REB                              43
AST                              10
STL                               7
BLK                               3
TOV                              16
PF                               16
PLUS_MINUS                  

There it is.

We can see the game was on April 4th, was in Toronto, and ended in an 18-point Raptors victory.
It can be confusing to read this, but the row is all relative to the Celtics (the team we queried).
All the stats (points, rebounds, blocks, plus/minus) are theirs.

If we wanted stats for both teams (a common use case), we'd need to get *both* rows for this game ID.
There are always two, one with stats for each team.
So back to the `LeagueGameFinder`!

In [14]:
game_id = last_raps_game.GAME_ID
game_id

'0021701171'

In [15]:
# Get **all** the games so we can filter to an individual GAME_ID
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]
# Find the game_id we want
full_game = all_games[all_games.GAME_ID == game_id]
full_game

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
10970,22017,1610612761,TOR,Toronto Raptors,0021701171,2018-04-04,TOR vs. BOS,W,240,96,40,92,0.435,10,33,0.303,6,8,0.750,12,36,48,23,10,6,10,25,18.0
10974,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,25,75,0.333,3,22,0.136,25,28,0.893,8,35,43,10,7,3,16,16,-18.0


Two rows, one with the Celtics' stats and one with the Raptors'.
You may want to join these these two rows into one, so you have stats for both teams in the same observation.

Because this is a common use case, I wrote a function for it.
This function will work for larger datasets too (even though we just have one game here);
you can run it on any game DataFrames.

In [16]:
import pandas as pd

def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(full_game)
game_df

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
1,22017,1610612761,TOR,Toronto Raptors,0021701171,2018-04-04,TOR vs. BOS,W,240,96,40,92,0.435,10,33,0.303,6,8,0.75,12,36,48,23,10,6,10,25,18.0,1610612738,BOS,Boston Celtics,BOS @ TOR,L,237,78,25,75,0.333,3,22,0.136,25,28,0.893,8,35,43,10,7,3,16,16,-18.0
